In [ ]:
import os
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
import pickle
import datetime
import matplotlib.ticker as ticker

import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot

In [ ]:
home = os.path.expanduser("~") 
os.chdir(str(project_dir))

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/operational-benchmarking/op-bench-runtime"
def get_write_time (algorithm, Ntx, Nty, target, nt, nr):
    command = base_cmd + " " + algorithm + " " + str(Ntx) + " " + str(Nty) + " " + str(target) + " " + str(nt) + " " + str(nr)
    return os.popen(command).read()

In [ ]:
trials = 1
reps = 10
Ntx = 32
Nty = [x for x in range(50, 92, 2)]
target = 1024
algorithms = ["BIRD-CPU-2D", "REDREC-CPU-V3-2D", "REDREC-GPU-V3-2D"]

for algorithm in algorithms:
    time_data  = []
    for i in range(len(Nty)):
        print(f"{i}/{len(Nty)}")
        write_time_list = []
        y = Nty[i]
        write_time_str = get_write_time(algorithm, Ntx, y, target, trials, reps)
        print(write_time_str)
        time_data += [(float(write_time_str.split(',')[0]), float(write_time_str.split(',')[1]))]
    # Saving the results 
    file_name = "timing-" + algorithm + "-target" + str(Ntx) + "x" + str(Ntx) + "-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
    with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name), "wb") as file:
        pickle.dump(
            {
                "xvals": Nty,
                "algorithm": algorithm,
                "experiment": file_name,
                "data": time_data
            }, file
        )
    

# Load results

In [ ]:
load_latest = False 
if not load_latest:
    file_name = "timing-REDREC-CPU-V3-2D-target32x32-2024-07-08, 09-23-32.pickle"

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name), "rb") as file:
    data = pickle.load(file)
    Nty = data["xvals"]
    time_data = data["data"]
    title = data["experiment"]

In [ ]:
xvals = waveforms_per_segment
yvals = [(time_data[i] * 1e6) for i in range(0, len(time_data), 2)]
yerr = [(time_data[i] * 1e6) for i in range(1, len(time_data), 2)]
runtimevals = [(xvals[i]*10) for i in range(0, len(xvals))]
runtimevals_err = [0 for i in range(0, len(xvals))]
fig, ax = plt.subplots() 
ax.set_xlabel("Number of waveforms per segment")
ax.set_ylabel("Time (us)")


prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yvals, runtimevals], yerr_list=[yerr, runtimevals_err], save_path=str(project_dir) + "/wfm-loading.pdf")
ax.legend(["Upload", "Stream"],fontsize=4)
ax.set_xlim(-2, 70)
ax.set_ylim(-100, 1000)
ax.set_yticks([0, 250, 500,750, 1000])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
ax.set_xticks([0, 4,8,16,32,64])
plt.savefig("wfm-loading.pdf", format='pdf', bbox_inches='tight')
plt.show()